# Claude Code Hooks System - Technical Reference

**Type**: Reference (Information-oriented)
**Last Updated**: 2025-09-21
**Version**: 1.0.0

## Overview

Complete technical reference for the Claude Code hooks system, including APIs, data structures, configuration schemas, and implementation details.

## Hook Configuration Schema

### settings.json Hook Structure

In [ ]:
# Complete hook configuration schema
HOOK_CONFIG_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "hooks": {
            "type": "object",
            "properties": {
                "Stop": {
                    "type": "array",
                    "items": {
                        "$ref": "#/definitions/hookEntry"
                    }
                },
                "SubagentStop": {
                    "type": "array",
                    "items": {
                        "$ref": "#/definitions/hookEntry"
                    }
                },
                "SessionEnd": {
                    "type": "array",
                    "items": {
                        "$ref": "#/definitions/hookEntry"
                    }
                },
                "PostToolUse": {
                    "type": "array",
                    "items": {
                        "$ref": "#/definitions/hookEntry"
                    }
                },
                "PreToolUse": {
                    "type": "array",
                    "items": {
                        "$ref": "#/definitions/hookEntry"
                    }
                }
            }
        }
    },
    "definitions": {
        "hookEntry": {
            "type": "object",
            "required": ["command", "args"],
            "properties": {
                "command": {
                    "type": "string",
                    "description": "Executable command (e.g., python, node)"
                },
                "args": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Command arguments"
                },
                "description": {
                    "type": "string",
                    "description": "Human-readable description"
                },
                "enabled": {
                    "type": "boolean",
                    "default": true,
                    "description": "Whether hook is enabled"
                },
                "timeout": {
                    "type": "integer",
                    "default": 120,
                    "description": "Timeout in seconds"
                }
            }
        }
    }
}

import json
print("Hook Configuration Schema:")
print(json.dumps(HOOK_CONFIG_SCHEMA, indent=2))

## Hook Input/Output Specifications

### Standard Hook Input Format

In [ ]:
# Hook input data structures
class HookInput:
    """Standard hook input structure passed via stdin."""
    
    # Stop/SubagentStop hook input
    STOP_INPUT = {
        "hook_type": "Stop",  # or "SubagentStop"
        "response_id": "uuid-string",
        "timestamp": "2025-09-21T00:00:00Z",
        "session_id": "session-uuid",
        "cwd": "/current/working/directory",
        "context": {
            "tools_used": ["Read", "Write", "Edit"],
            "files_modified": ["file1.py", "file2.py"],
            "response_length": 1234
        }
    }
    
    # PostToolUse hook input
    POST_TOOL_INPUT = {
        "hook_type": "PostToolUse",
        "tool_name": "Write",  # or "Edit", "MultiEdit", "NotebookEdit"
        "tool_input": {
            "file_path": "/path/to/file.py",
            "content": "file content (for Write)",
            "old_string": "text to replace (for Edit)",
            "new_string": "replacement text (for Edit)",
            "edits": [{  # for MultiEdit
                "old_string": "old",
                "new_string": "new"
            }]
        },
        "tool_result": {
            "success": True,
            "message": "File written successfully"
        }
    }
    
    # SessionEnd hook input
    SESSION_END_INPUT = {
        "hook_type": "SessionEnd",
        "session_id": "session-uuid",
        "start_time": "2025-09-21T00:00:00Z",
        "end_time": "2025-09-21T01:00:00Z",
        "duration_seconds": 3600,
        "statistics": {
            "total_responses": 10,
            "tools_used": {"Read": 5, "Write": 3, "Edit": 2},
            "files_modified": 8,
            "lines_changed": 150
        }
    }

print("Hook Input Specifications defined")
print("\nExample Stop hook input:")
import json
print(json.dumps(HookInput.STOP_INPUT, indent=2))

### Hook Output Requirements

In [ ]:
# Hook output specifications
class HookOutput:
    """Hook output requirements and formats."""
    
    # Exit codes
    EXIT_SUCCESS = 0  # Hook completed successfully
    EXIT_ERROR = 1    # Hook encountered error
    EXIT_SKIP = 2     # Hook skipped processing
    
    # Standard output format (to stdout)
    @staticmethod
    def format_log_message(message, level="INFO"):
        """Standard log message format."""
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return f"[{timestamp}] [{level}] {message}"
    
    # Quality check output file structure
    QUALITY_OUTPUT = {
        "timestamp": "2025-09-21T00:00:00Z",
        "response_id": "response-uuid",
        "hook_type": "Stop",
        "project_directory": "/path/to/project",
        "quality_checks": {
            "formatting": {
                "formatter": "black",
                "status": "success",  # or "error", "timeout"
                "files_formatted": ["file1.py", "file2.py"],
                "errors": [],
                "summary": "2 files need formatting"
            },
            "linting": {
                "linter": "ruff",
                "issues_found": 5,
                "issues_fixed": 3,
                "issues_remaining": 2
            },
            "complexity": {
                "high_complexity_functions": [
                    {"name": "complex_function", "complexity": 15, "file": "module.py"}
                ]
            }
        },
        "hook_info": {
            "hook_script": "/path/to/hook.py",
            "execution_time": "2025-09-21T00:00:00Z",
            "duration_ms": 1234
        }
    }

print("Hook Output Specifications:")
print(f"Success exit code: {HookOutput.EXIT_SUCCESS}")
print(f"Error exit code: {HookOutput.EXIT_ERROR}")
print(f"Skip exit code: {HookOutput.EXIT_SKIP}")
print("\nExample quality output structure:")
print(json.dumps(HookOutput.QUALITY_OUTPUT, indent=2))

## File Structure Reference

### Directory Layout

In [ ]:
# Hooks directory structure
HOOKS_DIRECTORY_STRUCTURE = """
.claude/hooks/
├── post-response-quality-check.py      # Quality check after responses
├── quality-remediation-trigger.py      # Orchestrate quality fixes
├── post-file-edit-unicode-cleanup.py   # Unicode cleanup after edits
├── session-end-quality-check.py        # Session-end quality report
├── tools/
│   ├── python_quality_manager.py       # Python quality management
│   ├── git_protection_manager.py       # Git protection workflows
│   ├── unicode_manager.py              # Unicode/emoji cleanup
│   ├── pattern_scanner.py              # Code pattern detection
│   ├── pattern_fixer.py                # Pattern auto-fixing
│   ├── ast_sast_analyzer.py            # AST-based analysis
│   └── safe_unicode_cleanup.py         # Safe Unicode operations
├── _concepts/
│   └── code_quality_auto_scan_fix.md   # Conceptual documentation
├── HANDOVER_DOCUMENTATION.md           # Session handover docs
├── CONTEXT_PRESERVATION.md             # Context preservation guide
├── SESSION_HANDOVER_SUMMARY.md         # Summary documentation
└── __pycache__/                        # Python cache files
"""

print("Hooks Directory Structure:")
print(HOOKS_DIRECTORY_STRUCTURE)

# Quality checks directory structure
QUALITY_DIRECTORY_STRUCTURE = """
.claude/quality-checks/
├── post-response-quality-*.json        # Quality check results
├── session-end-quality-*.json          # Session-end reports
├── hooks-trigger.log                   # Hook execution logs
├── triggers/
│   └── remediation-trigger-*.json      # Remediation trigger logs
└── remediation-plans/
    └── remediation-plan-*.json         # Remediation plans
"""

print("\nQuality Checks Directory Structure:")
print(QUALITY_DIRECTORY_STRUCTURE)

## API Reference: Python Quality Manager

In [ ]:
# PythonQualityManager API reference
class PythonQualityManagerAPI:
    """API reference for tools/python_quality_manager.py"""
    
    def __init__(self, project_dir="."):
        """Initialize quality manager.
        
        Args:
            project_dir: Path to project directory
        """
        pass
    
    def discover_pyproject_config(self):
        """Discover and parse pyproject.toml configuration.
        
        Returns:
            dict: Configuration from pyproject.toml or None
        """
        pass
    
    def check_compilation(self, file_path):
        """Check if Python file compiles.
        
        Args:
            file_path: Path to Python file
            
        Returns:
            dict: {"success": bool, "error": str or None}
        """
        pass
    
    def analyze_complexity(self, file_path):
        """Analyze code complexity using radon.
        
        Args:
            file_path: Path to Python file
            
        Returns:
            dict: Complexity metrics per function/class
        """
        pass
    
    def validate_docstrings(self, file_path):
        """Validate docstrings using pydocstyle.
        
        Args:
            file_path: Path to Python file
            
        Returns:
            list: Docstring violations found
        """
        pass
    
    def run_quality_check(self, file_path):
        """Run comprehensive quality check.
        
        Args:
            file_path: Path to Python file
            
        Returns:
            dict: Complete quality report
        """
        pass

print("PythonQualityManager API Reference defined")
print("\nKey methods:")
for method in dir(PythonQualityManagerAPI):
    if not method.startswith('_'):
        print(f"  - {method}()")

## API Reference: Git Protection Manager

In [ ]:
# GitProtectionManager API reference
class GitProtectionManagerAPI:
    """API reference for tools/git_protection_manager.py"""
    
    def __init__(self, repo_path="."):
        """Initialize git protection manager.
        
        Args:
            repo_path: Path to git repository
        """
        pass
    
    def create_protection_commit(self, operation_description, target_files=None):
        """Create a protection commit before automated changes.
        
        Args:
            operation_description: Description of planned operation
            target_files: List of files to be modified (None = all)
            
        Returns:
            dict: {"success": bool, "commit_hash": str, "errors": list}
        """
        pass
    
    def create_completion_commit(self, operation_description, tools_executed, 
                                files_modified, execution_summary):
        """Create a completion commit after successful automation.
        
        Args:
            operation_description: Description of completed operation
            tools_executed: List of tools that were run
            files_modified: List of files that were modified
            execution_summary: Summary statistics
            
        Returns:
            dict: {"success": bool, "commit_hash": str, "errors": list}
        """
        pass
    
    def rollback_to_protection_commit(self, commit_hash):
        """Rollback to a specific protection commit.
        
        Args:
            commit_hash: Hash of protection commit
            
        Returns:
            dict: {"success": bool, "errors": list}
        """
        pass
    
    def get_modified_files(self):
        """Get list of modified files in working directory.
        
        Returns:
            list: Paths of modified files
        """
        pass

print("GitProtectionManager API Reference defined")
print("\nCommit message formats:")
print("  Protection: 'feat: pre-automation protection commit - {description}'")
print("  Completion: 'fix: post-automation completion commit - {description}'")

## API Reference: Unicode Manager

In [ ]:
# UnicodeManager API reference
class UnicodeManagerAPI:
    """API reference for tools/unicode_manager.py"""
    
    # Unicode replacement mappings
    REPLACEMENTS = {
        # Arrows
        "→": "->",
        "←": "<-",
        "↑": "^",
        "↓": "v",
        # Checkmarks
        "✓": "[OK]",
        "✔": "[OK]",
        "✅": "[OK]",
        "❌": "[X]",
        # Bullets
        "•": "*",
        "▪": "*",
        "▫": "-",
        # Box drawing
        "─": "-",
        "│": "|",
        "┌": "+",
        "┐": "+",
        "└": "+",
        "┘": "+",
    }
    
    def __init__(self):
        """Initialize Unicode manager."""
        pass
    
    def process_file(self, file_path):
        """Process a file to remove/replace Unicode characters.
        
        Args:
            file_path: Path to file
            
        Returns:
            dict: {
                "processed": bool,
                "modified": bool,
                "unicode_replaced": int,
                "error": str or None
            }
        """
        pass
    
    def clean_text(self, text):
        """Clean Unicode from text string.
        
        Args:
            text: Text to clean
            
        Returns:
            tuple: (cleaned_text, replacement_count)
        """
        pass
    
    def is_safe_for_windows(self, text):
        """Check if text is safe for Windows console.
        
        Args:
            text: Text to check
            
        Returns:
            bool: True if safe, False if contains problematic Unicode
        """
        pass

print("UnicodeManager API Reference defined")
print(f"\nTotal replacement mappings: {len(UnicodeManagerAPI.REPLACEMENTS)}")
print("\nExample replacements:")
for char, replacement in list(UnicodeManagerAPI.REPLACEMENTS.items())[:5]:
    print(f"  '{char}' -> '{replacement}'")

## Environment Variables

In [ ]:
# Environment variables used by hooks
ENVIRONMENT_VARIABLES = {
    # Claude Code provided
    "CLAUDE_PROJECT_DIR": {
        "description": "Project directory path",
        "example": "/path/to/project",
        "provided_by": "Claude Code CLI"
    },
    "CLAUDE_TOOL_NAME": {
        "description": "Name of tool that triggered PostToolUse hook",
        "example": "Write",
        "provided_by": "Claude Code CLI"
    },
    "CLAUDE_SESSION_ID": {
        "description": "Current session identifier",
        "example": "session-123-uuid",
        "provided_by": "Claude Code CLI"
    },
    
    # Hook-specific
    "HOOK_DEBUG": {
        "description": "Enable debug output in hooks",
        "example": "1",
        "provided_by": "User"
    },
    "HOOK_TIMEOUT": {
        "description": "Override default hook timeout (seconds)",
        "example": "300",
        "provided_by": "User"
    },
    "QUALITY_MIN_PRIORITY": {
        "description": "Minimum priority for triggering remediation",
        "example": "medium",
        "provided_by": "User"
    },
    "UNICODE_CLEANUP_SKIP": {
        "description": "Skip Unicode cleanup even if configured",
        "example": "1",
        "provided_by": "User"
    }
}

print("Environment Variables Reference:")
for var_name, details in ENVIRONMENT_VARIABLES.items():
    print(f"\n{var_name}:")
    print(f"  Description: {details['description']}")
    print(f"  Example: {details['example']}")
    print(f"  Provided by: {details['provided_by']}")

## Configuration Files Reference

### pyproject.toml Integration

In [ ]:
# pyproject.toml configuration options
PYPROJECT_CONFIG = """
[tool.black]
line-length = 120
target-version = ['py311']
include = '\.pyi?$'
extend-exclude = '''
(
  /(  # directories
      \.git
    | \.hg
    | \.mypy_cache
    | \.tox
    | \.venv
    | _build
    | buck-out
    | build
    | dist
  )/
)
'''

[tool.isort]
profile = "black"
line_length = 120
multi_line_output = 3
include_trailing_comma = true
force_grid_wrap = 0
use_parentheses = true
ensure_newline_before_comments = true

[tool.ruff]
line-length = 120
select = [
    "E",   # pycodestyle errors
    "F",   # pyflakes
    "I",   # isort
    "N",   # pep8-naming
    "UP",  # pyupgrade
    "B",   # flake8-bugbear
    "C4",  # flake8-comprehensions
    "SIM", # flake8-simplify
]
ignore = [
    "E501", # line too long (handled by black)
    "B008", # do not perform function calls in argument defaults
]

[tool.pydocstyle]
convention = "google"
match-dir = "(?!test).*"
add-ignore = ["D107", "D212"]

[tool.radon]
complexity_threshold = 10  # Non-UI code
ui_complexity_threshold = 15  # UI components
"""

print("pyproject.toml Configuration Reference:")
print("=" * 60)
print(PYPROJECT_CONFIG)
print("=" * 60)

## Quality Metrics Reference

### Complexity Thresholds

In [ ]:
# Complexity metrics and thresholds
COMPLEXITY_METRICS = {
    "Cyclomatic Complexity": {
        "description": "Number of linearly independent paths through code",
        "thresholds": {
            "simple": "1-5",
            "moderate": "6-10",
            "complex": "11-20",
            "very_complex": ">20"
        },
        "recommended_max": {
            "non_ui": 10,
            "ui_component": 15
        }
    },
    "Cognitive Complexity": {
        "description": "How difficult code is to understand",
        "factors": [
            "Nested structures",
            "Flow-breaking statements",
            "Recursion",
            "Boolean operators"
        ],
        "recommended_max": 15
    },
    "Maintainability Index": {
        "description": "Composite metric of maintainability",
        "formula": "171 - 5.2 * ln(HV) - 0.23 * CC - 16.2 * ln(LOC)",
        "thresholds": {
            "high": ">80",
            "moderate": "40-80",
            "low": "<40"
        }
    }
}

print("Complexity Metrics Reference:")
for metric, details in COMPLEXITY_METRICS.items():
    print(f"\n{metric}:")
    print(f"  Description: {details['description']}")
    if 'thresholds' in details:
        print("  Thresholds:")
        for level, value in details['thresholds'].items():
            print(f"    - {level}: {value}")
    if 'recommended_max' in details:
        if isinstance(details['recommended_max'], dict):
            print("  Recommended maximum:")
            for context, value in details['recommended_max'].items():
                print(f"    - {context}: {value}")
        else:
            print(f"  Recommended maximum: {details['recommended_max']}")

## Git Commit Message Formats

In [ ]:
# Git commit message conventions
COMMIT_FORMATS = {
    "Protection Commit": {
        "format": "feat: pre-automation protection commit - {description}",
        "body": """Protected files before automated quality improvements
Operation: {operation_description}
Target files: {target_files_count} files
Timestamp: {timestamp}

This commit can be used for rollback if automation fails.
Hash: {commit_hash}""",
        "example": "feat: pre-automation protection commit - Python quality improvements - 5 quality issues"
    },
    "Completion Commit": {
        "format": "fix: post-automation completion commit - {description}",
        "body": """Automated quality improvements completed successfully
Tools executed: {tools_list}
Files modified: {files_count}
Issues resolved: {issues_count}

All files compile successfully after automation.
Protection commit: {protection_hash}""",
        "example": "fix: post-automation completion commit - Python quality improvements - 139 quality issues resolved"
    },
    "Manual Quality Fix": {
        "format": "fix({scope}): {description}",
        "body": """Manual quality improvements
- {change_1}
- {change_2}
- {change_3}""",
        "example": "fix(quality): resolve black formatting issues in 5 files"
    }
}

print("Git Commit Message Formats:")
for commit_type, details in COMMIT_FORMATS.items():
    print(f"\n{commit_type}:")
    print(f"  Format: {details['format']}")
    print(f"  Example: {details['example']}")
    print("  Body template:")
    for line in details['body'].split('\n'):
        print(f"    {line}")

## Performance Specifications

### Hook Performance Requirements

In [ ]:
# Performance specifications and limits
PERFORMANCE_SPECS = {
    "timeouts": {
        "post_response_quality": 60,  # seconds
        "quality_remediation": 120,   # seconds
        "unicode_cleanup": 30,         # seconds
        "session_end_quality": 180,    # seconds
        "git_operations": 30,          # seconds
        "compilation_check": 10        # seconds per file
    },
    "file_limits": {
        "max_file_size_mb": 1,          # Skip files larger than this
        "max_files_per_check": 100,      # Maximum files to check at once
        "recent_files_window_minutes": 5 # Only check files modified within this window
    },
    "memory_limits": {
        "max_log_file_size_mb": 10,      # Rotate logs after this size
        "max_quality_files_kept": 100,   # Clean up old quality check files
        "max_remediation_plans_kept": 50 # Clean up old remediation plans
    },
    "performance_targets": {
        "hook_startup_ms": 100,          # Hook should start within this time
        "file_processing_ms_per_kb": 10, # Processing time per KB of file
        "git_commit_ms": 500,            # Git commit operation time
        "quality_check_ms_per_file": 200 # Quality check per file
    }
}

print("Performance Specifications:")
for category, specs in PERFORMANCE_SPECS.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    for spec, value in specs.items():
        unit = "seconds" if "timeout" in category else \
                "MB" if "mb" in spec else \
                "ms" if "ms" in spec else \
                "minutes" if "minutes" in spec else ""
        print(f"  {spec}: {value} {unit}")

## Error Codes and Messages

In [ ]:
# Standard error codes and messages
ERROR_CODES = {
    # Hook errors (1000-1999)
    1000: "Hook initialization failed",
    1001: "Invalid hook input JSON",
    1002: "Hook timeout exceeded",
    1003: "Hook permission denied",
    1004: "Hook dependency missing",
    
    # Quality check errors (2000-2999)
    2000: "Quality check failed",
    2001: "Black formatter error",
    2002: "Isort error",
    2003: "Ruff linter error",
    2004: "Complexity check failed",
    2005: "Docstring validation failed",
    
    # Git errors (3000-3999)
    3000: "Git operation failed",
    3001: "Git repository not found",
    3002: "Git uncommitted changes",
    3003: "Git protection commit failed",
    3004: "Git rollback failed",
    3005: "Git completion commit failed",
    
    # Compilation errors (4000-4999)
    4000: "Python compilation failed",
    4001: "Syntax error in Python file",
    4002: "Import error in Python file",
    4003: "Indentation error in Python file",
    
    # File operation errors (5000-5999)
    5000: "File operation failed",
    5001: "File not found",
    5002: "File permission denied",
    5003: "File too large",
    5004: "Invalid file format",
    5005: "Unicode decode error"
}

print("Error Codes Reference:")
categories = {
    "Hook Errors (1000-1999)": range(1000, 2000),
    "Quality Check Errors (2000-2999)": range(2000, 3000),
    "Git Errors (3000-3999)": range(3000, 4000),
    "Compilation Errors (4000-4999)": range(4000, 5000),
    "File Operation Errors (5000-5999)": range(5000, 6000)
}

for category, code_range in categories.items():
    print(f"\n{category}:")
    for code in code_range:
        if code in ERROR_CODES:
            print(f"  {code}: {ERROR_CODES[code]}")

## Summary

This technical reference provides complete specifications for the Claude Code hooks system:

### Key Components
- **Configuration Schema**: Complete JSON schema for hook configuration
- **Input/Output Specs**: Data structures for hook communication
- **API References**: Detailed API documentation for all manager classes
- **Environment Variables**: All environment variables used by hooks
- **Error Codes**: Standardized error codes and messages

### Implementation Standards
- **Performance**: Defined timeouts and processing limits
- **Quality Metrics**: Complexity thresholds and quality standards
- **Git Conventions**: Standardized commit message formats
- **File Structure**: Organized directory layout

### Integration Points
- **Claude Code CLI**: Hook triggers and environment setup
- **Python Tools**: Black, isort, ruff, pydocstyle integration
- **Git Workflows**: Protection and completion commit patterns
- **Quality Reports**: Standardized JSON output formats

This reference serves as the authoritative source for implementing, extending, or troubleshooting the Claude Code hooks system.